# Using AdaptiveMasterMSM to run Gromacs
A **gro** file corresponding to alanine dipeptide is employed as example. Below the equilibration of the system is done.

In [1]:
from adaptivemastermsm import controller

We invoke **Controller** class for the **Equilibration** step, since later this will be used for production.

In [2]:
con = controller.Controller()
con.run_parameters('Equilibration')

In [3]:
from adaptivemastermsm.system import system

**System** class must be started to specify everything related to the system.

In [4]:
alaTB = system.System('logs/alaTB.gro', forcefield='amber03', water='tip3p')

In [5]:
alaTB.gen_top()

gmx pdb2gmx -f logs/alaTB.gro -o conf.gro -p topol.top -ff amber03 -water tip3p


In [6]:
# note parameter 'd' is passed through controller.run dictionary,
# and similarly for other parameters below
dist = con.run['box_size']
print(dist)
alaTB.build_box(d=dist)

1.0
gmx editconf -f conf.gro -o conf_edit.gro -c -d 1.000000 -bt cubic


In [7]:
alaTB.solvate()

gmx solvate -cp conf_edit.gro -cs spc216.gro -p topol.top -o conf_solv.gro


In [8]:
alaTB.gro

'conf.gro'

In [9]:
an = con.run['Cl']
cat = con.run['Na']
print(an,cat)
alaTB.ionize(anion=an, cation=cat)

0 0
gmx grompp -f data/minimization.mdp -c conf_solv.gro -p topol.top -o ions.tpr; gmx genion -s ions.tpr -o conf_solv_ions.gro -p topol.top 


(b'Analysing residue names:\nThere are:     3    Protein residues\nThere are:   667      Water residues\nAnalysing Protein...\nThis run will generate roughly 14 Mb of data\n',
 b'              :-) GROMACS - gmx grompp, 2020.1-Ubuntu-2020.1-1 (-:\n\n                            GROMACS is written by:\n     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen\n    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    \n Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     \n  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  \n  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis\n    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    \n  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   \n    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   \n    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvet

Minimization options are specified by **minimization.mdp** file.

In [10]:
alaTB.minimize()

gmx grompp -f data/minimization.mdp -c conf_solv_ions.gro -p topol.top -o minimization; gmx mdrun -v -s minimization -x minimization.xtc -deffnm minimization


(b'Analysing residue names:\nThere are:     3    Protein residues\nThere are:   667      Water residues\nAnalysing Protein...\nThis run will generate roughly 14 Mb of data\n',
 b'              :-) GROMACS - gmx grompp, 2020.1-Ubuntu-2020.1-1 (-:\n\n                            GROMACS is written by:\n     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen\n    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    \n Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     \n  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  \n  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis\n    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    \n  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   \n    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   \n    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvet

Obtain a stable thermodynamic ensemble by **NVT** and, consecutively, **NPT** equilibration.

In [11]:
alaTB.nvt()

gmx grompp -f nvt.mdp -c minimization.gro -r minimization.gro -p topol.top -o nvt.tpr; gmx mdrun -v -s nvt.tpr -deffnm nvt


(b'turning H bonds into constraints...\nturning H bonds into constraints...\nAnalysing residue names:\nThere are:     3    Protein residues\nThere are:   667      Water residues\nAnalysing Protein...\nDetermining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K\nCalculated rlist for 1x1 atom pair-list as 1.035 nm, buffer size 0.035 nm\nSet rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm\nNote that mdrun will redetermine rlist based on the actual pair-list setup\nCalculating fourier grid dimensions for X Y Z\nUsing a fourier grid of 20x20x20, spacing 0.137 0.137 0.137\nThis run will generate roughly 0 Mb of data\n',
 b'              :-) GROMACS - gmx grompp, 2020.1-Ubuntu-2020.1-1 (-:\n\n                            GROMACS is written by:\n     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen\n    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    \n Aldert van Buuren   Rudi van Drunen     Anton Feenstra       

In [12]:
alaTB.npt()

gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr; gmx mdrun -v -s npt.tpr -deffnm npt


(b'turning H bonds into constraints...\nturning H bonds into constraints...\nAnalysing residue names:\nThere are:     3    Protein residues\nThere are:   667      Water residues\nAnalysing Protein...\nDetermining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K\nCalculated rlist for 1x1 atom pair-list as 1.035 nm, buffer size 0.035 nm\nSet rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm\nNote that mdrun will redetermine rlist based on the actual pair-list setup\nCalculating fourier grid dimensions for X Y Z\nUsing a fourier grid of 20x20x20, spacing 0.137 0.137 0.137\nThis run will generate roughly 0 Mb of data\n',
 b'              :-) GROMACS - gmx grompp, 2020.1-Ubuntu-2020.1-1 (-:\n\n                            GROMACS is written by:\n     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen\n    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    \n Aldert van Buuren   Rudi van Drunen     Anton Feenstra       